In [ ]:
import numpy as np
import cv2 as cv
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt
from random import randint
from math import *

In [ ]:
def mask(img, choice):
    # Gaussian blur
    kernel_size = 21
    blur_img = cv.GaussianBlur(img,(kernel_size, kernel_size),0)

    if choice == 'blue':
        # lower = np.array([100,30,16])
        # upper = np.array([250,148,255])
        lower = np.array([90, 100, 12])
        upper = np.array([140, 255, 235])
    elif choice == 'white':
        lower = np.array([0,0,0])
        upper = np.array([0,0,255])
    elif choice == 'black':
        lower = np.array([0,0,0])
        upper = np.array([50,50,100])

    image = cv.cvtColor(blur_img,cv.COLOR_BGR2HSV)
    mask = cv.inRange(image, lower, upper)
    return mask

In [ ]:
def lerp(A, B, ratio):
    return (A * ratio) + ((1 - ratio) * B)

In [ ]:
def get_rect_direction(rect):
    box = cv.boxPoints(rect)
    start = lerp(box[0], box[3], 0.5)
    end = lerp(box[1], box[2], 0.5)
    angle = 90 - rect[2] 
    print("length : ", np.linalg.norm(box[0] - box[3]))
    if np.linalg.norm(box[0] - box[3]) > np.linalg.norm(box[0] - box[1]):
        start = lerp(box[0], box[1], 0.5)
        end = lerp(box[2], box[3], 0.5)
        angle += 90
    print("angle : ", angle)

    return start, end, angle

In [ ]:
def rotate_vector(vec, angle_deg, anchor=None):
    theta = np.deg2rad(angle_deg)
    rot = np.array([
        [cos(theta), -sin(theta)],
        [sin(theta), cos(theta)]]    
    )
    if anchor is None:
        anchor = (0,0)
    return np.dot(rot, vec - anchor) + anchor

In [ ]:
cap = cv.VideoCapture(0)
if not cap.isOpened():
    print("Cannot open camera")
    exit()

cap.set(cv.CAP_PROP_FRAME_HEIGHT, 240)
cap.set(cv.CAP_PROP_FRAME_WIDTH, 352)
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    # Our operations on the frame come here
    processed_img = mask(frame, 'blue')
    # Display the resulting frame
    cv.imshow('frame', processed_img)
    if cv.waitKey(1) == ord('q'):
        break
# When everything done, release the capture
cap.release()
cv.destroyAllWindows()

In [ ]:
stream = cv.VideoCapture(0)
if not stream.isOpened():
    print("Cannot open camera")
    exit(-1)
stream.set(cv.CAP_PROP_FRAME_HEIGHT, 240)
stream.set(cv.CAP_PROP_FRAME_WIDTH, 352)

color = (randint(0,256), randint(0,256), randint(0,256))

while stream.isOpened():
    ret, frame = stream.read()
    if not ret:
        print("Cannot read frame from camera")
        exit(-1)

    #Crop l'image
    # frame = frame[frame.shape[0] // 2:, :]scipy homogeneous transformation

    # Masquage de l'image par rapport àà un couleur
    m = mask(frame, "blue")
    cv.imshow('mask', m)

    # Récupération des contours
    contours, hierarchy = cv.findContours(m, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    cv.drawContours(frame, contours, -1, (0,255,0), 1)

    cv.imshow('frame', frame)
    if len(contours) > 0:
        # On considère le contour le plus étendu et on calcule une bounding box orientée
        countour = max(contours, key=cv.contourArea)
        minRect = cv.minAreaRect(countour)
        drawing = np.zeros((frame.shape[0], frame.shape[1], 3), dtype=np.uint8)

        start, end, angle = get_rect_direction(minRect)
        cv.line(drawing, np.int0(start), np.int0(end), color)
        cv.imshow("boxes", drawing)

    if cv.waitKey(1) == ord('q'):
        break
stream.release()
cv.destroyAllWindows()

In [ ]:
stream.release()
stream = cv.VideoCapture(0)

if not stream.isOpened():
    print("Cannot open camera")
    exit(-1)
    
stream.set(cv.CAP_PROP_FRAME_HEIGHT, 240)
stream.set(cv.CAP_PROP_FRAME_WIDTH, 352)


ret, frame = stream.read()
if not ret:
    print("Cannot read frame from camera")
    exit(-1)

cv.imwrite("benchmark.png", frame)